In [ ]:
!pip install tensorflow tensorflow_datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the CIFAR-10 dataset
dataset, info = tfds.load('cifar10', split='train', with_info=True)

# Preprocess the dataset
def preprocess_data(data):
    image = tf.image.resize(data['image'], (224, 224))  # Resize image to match model input shape
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(data['label'], depth=10)  # Convert label to one-hot encoding
    return image, label

# Apply preprocessing to the dataset
train_dataset = dataset.map(preprocess_data)

# Split the dataset into training and validation sets
train_size = int(0.8 * info.splits['train'].num_examples)
train_dataset = train_dataset.take(train_size)
val_dataset = train_dataset.skip(train_size)

# Configure batch size and prefetching
batch_size = 2
train_dataset = train_dataset.batch(batch_size).prefetch(1)
val_dataset = val_dataset.batch(batch_size).prefetch(1)






In [ ]:
# Load the MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224, 224, 3))  # Update input shape

# Freeze the base model's layers
base_model.trainable = False

# Add a global average pooling layer
global_average_pooling = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

# Add a fully connected output layer with softmax activation for classification
output_layer = tf.keras.layers.Dense(10, activation='softmax')(global_average_pooling)

# Create the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=output_layer)

# Define loss function and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
metrics = [tf.keras.metrics.CategoricalAccuracy()]

# Compile the model
model.compile(optimizer='adam', loss=loss_fn, metrics=metrics)


In [ ]:
# Train the model
epochs = 1
model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

In [ ]:
# Save the trained model
model.save('mobilenetv2_cifar10_model.h5')